<a href="https://colab.research.google.com/github/abhi-11nav/Airline-delay-prediction/blob/main/Airline_delay_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Importing the libraries for dataset import and manipulation

import pandas as pd 
import numpy as np 

In [13]:
# link for raw data uploaded on github

dataset_path = "https://raw.githubusercontent.com/abhi-11nav/Airline-delay-prediction/main/Airlines.csv"

In [14]:
# Loading the dataset into the dataset variable

dataset = pd.read_csv(dataset_path)

In [15]:
# Printing the first 5 rows of the dataset

dataset.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [16]:
# Printing the shape of data

dataset.shape

(539383, 9)

# DATA PREPROCESSING 

In [27]:
# Checking for missing values in the dataset 

missing_val_feature_count = 0

for col in dataset.columns:
  if dataset[col].isna().any() == True:
    missing_val_feature_count += 1

if missing_val_feature_count>0:
  print("WARNING - There are missing values in the dataset")
else:
  print("There are no missing values in the dataset")

There are no missing values in the dataset


This shows us that there are no missing values in the dataset

In [28]:
# Removing unnecessary feature "id" because it consists of all unique values

dataset.drop("id", axis=1,inplace=True)

"id" feature has been removed from the dataset

axis = 1 is to specify the axis to search on for removal of the feature,

inplace = True for removing the columns permanently from the data

.

**CLASSIFYING FEATURES AND LABELS**

DELAY IS OUR DEPENDENT VARIABLE (LABEL/Y)

REAMINING FEATURES ARE OUR INDEPENDENT VARIABLES ( FEATURES/X)

In [34]:
# Let us focus on the categorical variables of the dataset

# Printing the number of categorical variables
categorical_features = []

for col in dataset.columns:
  if type(dataset[col][0]) == str:
    categorical_features.append(col)

print(categorical_features,"are the categorical features in the dataset")

['Airline', 'AirportFrom', 'AirportTo'] are the categorical features in the dataset


In [36]:
# Now that we know how many categorical features are there in the dataset. Let us extract more information about it. 

unique_value_count = {}

for columns in categorical_features:
  unique_value_count[columns] = len(dataset[columns].value_counts())

print(unique_value_count)

{'Airline': 18, 'AirportFrom': 293, 'AirportTo': 293}


Airline feature has 18 unique values which is fine but the other two features "AirportTo" and "AirportFrom" has 293 unique values which have to be dealth with. 